In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing, metrics, linear_model
from nltk import word_tokenize
from nltk.corpus import stopwords
from keras.utils import np_utils
stop_words = stopwords.words('english')

Using TensorFlow backend.
/u/74/surikua1/unix/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [3]:
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping

In [14]:
from gensim.scripts.glove2word2vec import glove2word2vec
glove2word2vec(glove_input_file="glove.840B.300d.txt", word2vec_output_file="gensim_glove_vectors.txt")

(2196017, 300)

In [4]:
from gensim.models.keyedvectors import KeyedVectors
glove_model = KeyedVectors.load_word2vec_format("gensim_glove_vectors.txt", binary=False)

In [5]:
def sent2vec(s):
    words = str(s).lower()
    words = word_tokenize(words)
    words = [w for w in words if not w in stop_words]
    words = [w for w in words if w.isalpha()]
    M = []
    for w in words:
        try:
            M.append(glove_model[w])
        except:
            continue
    M = np.array(M)
    v = M.sum(axis=0)
    if type(v) != np.ndarray:
        return np.zeros(300)
    return v / np.sqrt((v ** 2).sum())

In [6]:
train_data = pd.read_csv("train.csv")
print(f'{train_data.shape} train data shape')

lbl_enc = preprocessing.LabelEncoder()
y = lbl_enc.fit_transform(train_data.author.values)

xtrain, xvalid, ytrain, yvalid = train_test_split(train_data.text.values, y, 
                                                  stratify=y, 
                                                  random_state=42, 
                                                  test_size=0.1, shuffle=True)

(19579, 3) train data shape


In [7]:
xtrain_glove = [sent2vec(x) for x in xtrain]
xvalid_glove = [sent2vec(x) for x in xvalid]

In [8]:
xtrain_glove = np.array(xtrain_glove)
xvalid_glove = np.array(xvalid_glove)

In [43]:
# Fitting a simple xgboost on glove features
clf = linear_model.LogisticRegression(C = 1.0)
clf.fit(xtrain_glove, ytrain)
predictions = clf.predict_proba(xvalid_glove)

print ("logloss: %0.3f " % metrics.log_loss(yvalid, predictions))

logloss: 0.732 


In [9]:
# scale the data before any neural net:
scl = preprocessing.StandardScaler()
xtrain_glove_scl = scl.fit_transform(xtrain_glove)
xvalid_glove_scl = scl.transform(xvalid_glove)

In [10]:
# we need to binarize the labels for the neural net
ytrain_enc = np_utils.to_categorical(ytrain)
yvalid_enc = np_utils.to_categorical(yvalid)

In [11]:
# using keras tokenizer here
token = text.Tokenizer(num_words=None)
max_len = 70

token.fit_on_texts(list(xtrain) + list(xvalid))
xtrain_seq = token.texts_to_sequences(xtrain)
xvalid_seq = token.texts_to_sequences(xvalid)

# zero pad the sequences
xtrain_pad = sequence.pad_sequences(xtrain_seq, maxlen=max_len)
xvalid_pad = sequence.pad_sequences(xvalid_seq, maxlen=max_len)

word_index = token.word_index

In [12]:
# create an embedding matrix for the words we have in the dataset
embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in word_index.items():
    if word not in glove_model:
        continue
    embedding_vector = glove_model[word]
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [13]:
# A simple LSTM with glove embeddings and two dense layers
model = Sequential()
model.add(Embedding(len(word_index) + 1,
                     300,
                     weights=[embedding_matrix],
                     input_length=max_len,
                     trainable=False))
model.add(SpatialDropout1D(0.3))
model.add(LSTM(100, dropout=0.3, recurrent_dropout=0.3))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(3))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [14]:
model.fit(xtrain_pad, y=ytrain_enc, batch_size=512, epochs=100, verbose=1, validation_data=(xvalid_pad, yvalid_enc))

Train on 17621 samples, validate on 1958 samples
Epoch 1/100
17621/17621 [==============================] - 6s 334us/step - loss: 1.0828 - val_loss: 0.9984
Epoch 2/100
17621/17621 [==============================] - 5s 272us/step - loss: 0.9726 - val_loss: 0.8494
Epoch 3/100
17621/17621 [==============================] - 4s 241us/step - loss: 0.8889 - val_loss: 0.7694
Epoch 4/100
17621/17621 [==============================] - 4s 228us/step - loss: 0.8421 - val_loss: 0.7340
Epoch 5/100
17621/17621 [==============================] - 4s 240us/step - loss: 0.8239 - val_loss: 0.7262
Epoch 6/100
17621/17621 [==============================] - 4s 249us/step - loss: 0.7971 - val_loss: 0.6972
Epoch 7/100
17621/17621 [==============================] - 4s 252us/step - loss: 0.7666 - val_loss: 0.6750
Epoch 8/100
17621/17621 [==============================] - 4s 253us/step - loss: 0.7541 - val_loss: 0.6588
Epoch 9/100
17621/17621 [==============================] - 4s 204us/step - loss: 0.7396 - val_l